# Assess wildfire damage with Amazon SageMaker geospatial capabilities

This notebook demonstrates how to use Amazon SageMaker geospatial capabilities to assess wildfire damages using multi-temporal Sentinel-2 satellite data.

The area of interest for this example is located in Northern California, from a region which was affected by the [Dixie Wildfire](https://en.wikipedia.org/wiki/Dixie_Fire) in 2021.

The workflow is as follows:

- Step 1: [Import SageMaker geospatial capabilities SDK](#Import-SageMaker-geospatial-capabilities-SDK)
- Step 2: [Inspect the area of interest](#Inspect-the-area-of-interest)
- Step 3: [Create an Earth Observation Job (EOJ) to perform landcover segmentation](#Create-an-Earth-Observation-Job-to-perform-landcover-segmentation)
- Step 4: [Visualize EOJ results in Amazon SageMaker geospatial Map SDK](#Visualize-EOJ-results-in-Amazon-SageMaker-geospatial-Map-SDK)
- Step 5: [Export EOJ output to S3](#Export-EOJ-output-to-S3)
- Step 6: [Quantify loss of vegetation and wildfire impact area](#Quantify-loss-of-vegetation-and-wildfire-impact-area)

## Prerequisites

This notebook runs with Kernel Geospatial 1.0. Note that the following policies need to be attached to the execution role that you used to run this notebook:

- AmazonSageMakerFullAccess
- AmazonSageMakerGeospatialFullAccess

You can see the policies attached to the role in the IAM console under the permissions tab. If required, add the roles using the 'Add Permissions' button.

In addition to these policies, ensure that the execution role's trust policy allows the SageMaker-GeoSpatial service to assume the role. This can be done by adding the following trust policy using the 'Trust relationships' tab:

```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": [
                    "sagemaker.amazonaws.com",
                    "sagemaker-geospatial.amazonaws.com"
                ]
            },
            "Action": "sts:AssumeRole"
        }
    ]
}
```

## Import SageMaker geospatial capabilities SDK

In [ ]:
import boto3
import sagemaker
import sagemaker_geospatial_map

session = boto3.Session()
execution_role = sagemaker.get_execution_role()
geospatial_client = session.client(service_name="sagemaker-geospatial")

## Inspect the area of interest

The raster data collection is queried with coordinates of the area impacted by the wildfire, and returns a list of satellite imagery matching the selected filters.

The data in cloud optimized GeoTIFF (COG) format allows a visual inspection of the impacted area before and after the wildfire.

In [ ]:
search_params = {
    "Arn": "arn:aws:sagemaker-geospatial:us-west-2:378778860802:raster-data-collection/public/nmqj48dcu3g7ayw8",  # Sentinel-2 L2A data
    "RasterDataCollectionQuery": {
        "AreaOfInterest": {
            "AreaOfInterestGeometry": {
                "PolygonGeometry": {
                    "Coordinates": [
                        [
                            [-121.32559295351282, 40.386534879495315],
                            [-121.32559295351282, 40.09770246706907],
                            [-120.86738632168885, 40.09770246706907],
                            [-120.86738632168885, 40.386534879495315],
                            [-121.32559295351282, 40.386534879495315],
                        ]
                    ]
                }
            }
        },
        "TimeRangeFilter": {
            "StartTime": "2021-06-01T00:00:00Z",
            "EndTime": "2021-09-30T23:59:59Z",
        },
        "PropertyFilters": {
            "Properties": [{"Property": {"EoCloudCover": {"LowerBound": 0, "UpperBound": 0.1}}}],
            "LogicalOperator": "AND",
        },
        "BandFilter": ["visual"],
    },
}

cog_urls = []
next_token = True
while next_token:
    search_result = geospatial_client.search_raster_data_collection(**search_params)
    for item in search_result["Items"]:
        asset_url = item["Assets"]["visual"]["Href"]
        cog_urls.append(asset_url)
    next_token = search_result.get("NextToken")
    search_params["NextToken"] = next_token

In [ ]:
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt

cog_urls.sort(key=lambda x: x.split("TFK_")[1])

src_pre = rasterio.open(cog_urls[0])
src_post = rasterio.open(cog_urls[-1])

fig, (ax_before, ax_after) = plt.subplots(1, 2, figsize=(14, 7))
subplot = show(src_pre, ax=ax_before)
subplot.axis("off")
subplot.set_title("Pre-wildfire ({})".format(cog_urls[0].split("TFK_")[1]))
subplot = show(src_post, ax=ax_after)
subplot.axis("off")
subplot.set_title("Post-wildfire ({})".format(cog_urls[-1].split("TFK_")[1]))
plt.show()

## Create an Earth Observation Job to perform landcover segmentation

The following cell shows how to launch an Earth Observation Job (EOJ). In this example, a pre-trained machine learning model for land cover segmentation is used. Depending on your use case, you can choose from a variety of operations and models when running an EOJ.

In addition to the type of operation, you can also select the area of interest, choose the data providers, and set time-range based and cloud coverage percentage filters.

In [ ]:
eoj_input_config = {
    "RasterDataCollectionQuery": {
        "RasterDataCollectionArn": "arn:aws:sagemaker-geospatial:us-west-2:378778860802:raster-data-collection/public/nmqj48dcu3g7ayw8",
        "AreaOfInterest": {
            "AreaOfInterestGeometry": {
                "PolygonGeometry": {
                    "Coordinates": [
                        [
                            [-121.32559295351282, 40.386534879495315],
                            [-121.32559295351282, 40.09770246706907],
                            [-120.86738632168885, 40.09770246706907],
                            [-120.86738632168885, 40.386534879495315],
                            [-121.32559295351282, 40.386534879495315],
                        ]
                    ]
                }
            }
        },
        "TimeRangeFilter": {
            "StartTime": "2021-06-01T00:00:00Z",
            "EndTime": "2021-09-30T23:59:59Z",
        },
        "PropertyFilters": {
            "Properties": [{"Property": {"EoCloudCover": {"LowerBound": 0, "UpperBound": 0.1}}}],
            "LogicalOperator": "AND",
        },
    }
}

eoj_config = {"LandCoverSegmentationConfig": {}}

response = geospatial_client.start_earth_observation_job(
    Name="dixie-wildfire-landcover-2021",
    InputConfig=eoj_input_config,
    JobConfig=eoj_config,
    ExecutionRoleArn=execution_role,
)
eoj_arn = response["Arn"]
eoj_arn

In [ ]:
import time
import datetime

# check status of created Earth Observation Job and wait until it is completed
eoj_completed = False
while not eoj_completed:
    response = geospatial_client.get_earth_observation_job(Arn=eoj_arn)
    print(
        "Earth Observation Job status: {} (Last update: {})".format(
            response["Status"], datetime.datetime.now()
        ),
        end="\r",
    )
    eoj_completed = True if response["Status"] == "COMPLETED" else False
    if not eoj_completed:
        time.sleep(30)

## Visualize EOJ results in Amazon SageMaker geospatial Map SDK

The following cells show how to create a embedded map instance with the geospatial Map SDK and visualize input and output of the Earth Observation Job in the map.

In [ ]:
Map = sagemaker_geospatial_map.create_map({"is_raster": True})
Map.set_sagemaker_geospatial_client(geospatial_client)

# Render the map
Map.render()

In [ ]:
time_range_filter = {
    "start_date": "2021-06-01T00:00:00Z",
    "end_date": "2021-09-30T23:59:59Z",
}

# Visualize input
config = {"label": "Input"}
input_layer = Map.visualize_eoj_input(
    Arn=eoj_arn, config=config, time_range_filter=time_range_filter
)

# Visualize output
config = {"preset": "singleBand", "band_name": "mask"}
output_layer = Map.visualize_eoj_output(
    Arn=eoj_arn, config=config, time_range_filter=time_range_filter
)

#### Land Cover Segmentation Visualization Legend

![Legend for Land Cover Segmentation](https://docs.aws.amazon.com/images/sagemaker/latest/dg/images/geo_landcover_ss.png)


## Export EOJ output to S3

In [ ]:
sagemaker_session = sagemaker.Session()
export_bucket = (
    sagemaker_session.default_bucket()
)  # Alternatively you can use your custom bucket here.
bucket_prefix = "eoj_dixie_wildfire_landcover"

response = geospatial_client.export_earth_observation_job(
    Arn=eoj_arn,
    ExecutionRoleArn=execution_role,
    OutputConfig={"S3Data": {"S3Uri": f"s3://{export_bucket}/{bucket_prefix}/", "KmsKeyId": ""}},
)

# Wait until EOJ has been exported to S3
while not response["ExportStatus"] == "SUCCEEDED":
    response = geospatial_client.get_earth_observation_job(Arn=eoj_arn)
    print(
        "Export of Earth Observation Job status: {} (Last update: {})".format(
            response["ExportStatus"], datetime.datetime.now()
        ),
        end="\r",
    )
    if not response["ExportStatus"] == "SUCCEEDED":
        time.sleep(30)

## Quantify loss of vegetation and wildfire impact area

The following cells show how the exported EOJ data can be processed further to quantify the vegetation loss caused by the wildfire and visualize the area which has been impacted.

In [ ]:
import os
from glob import glob

s3_bucket = session.resource("s3").Bucket(export_bucket)

# download land cover masks from S3 bucket
mask_dir = "./dixie-wildfire-landcover/masks"
os.makedirs(mask_dir, exist_ok=True)
for s3_object in s3_bucket.objects.filter(Prefix=bucket_prefix).all():
    path, filename = os.path.split(s3_object.key)
    if "output" in path:
        mask_local_path = mask_dir + "/" + filename
        s3_bucket.download_file(s3_object.key, mask_local_path)
        print("Downloaded mask: " + mask_local_path)

mask_files = glob(os.path.join(mask_dir, "*.tif"))
mask_files.sort(key=lambda x: x.split("TFK_")[1])

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors
import matplotlib.patches as mpatches
import numpy as np
import tifffile

landcover_simple_colors = {
    "not vegetated": "khaki",
    "vegetated": "olivedrab",
    "water": "lightsteelblue",
}


def extract_masks(date_str):
    mask_file = list(filter(lambda x: date_str in x, mask_files))[0]
    mask = tifffile.imread(mask_file)
    focus_area_mask = mask[400:1100, 600:1350]

    vegetation_mask = np.isin(focus_area_mask, [4]).astype(np.uint8)
    water_mask = np.isin(focus_area_mask, [6]).astype(np.uint8)
    water_mask[water_mask > 0] = 2
    additive_mask = np.add(vegetation_mask, water_mask).astype(np.uint8)

    return (focus_area_mask, vegetation_mask, additive_mask)


masks_20210603 = extract_masks("20210603")
masks_20210926 = extract_masks("20210926")

### Visualize difference in vegetation before and after the wildfire

In [ ]:
fig = plt.figure(figsize=(14, 7))

fig.add_subplot(1, 2, 1)
plt.imshow(
    masks_20210603[2],
    cmap=matplotlib.colors.ListedColormap(list(landcover_simple_colors.values()), N=None),
)
plt.title("Pre-wildfire")
plt.axis("off")
ax = fig.add_subplot(1, 2, 2)
hs = plt.imshow(
    masks_20210926[2],
    cmap=matplotlib.colors.ListedColormap(list(landcover_simple_colors.values()), N=None),
)
plt.title("Post-wildfire")
plt.axis("off")
patches = [mpatches.Patch(color=i[1], label=i[0]) for i in landcover_simple_colors.items()]
plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0)
plt.show()

### Quantify of loss in vegetation caused by wildfire

In [ ]:
vegetation_loss = round((1 - (masks_20210926[1].sum() / masks_20210603[1].sum())) * 100, 2)
diff_mask = np.add(masks_20210603[1], masks_20210926[1])
plt.figure(figsize=(6, 6))
plt.title("Loss in vegetation ({}%)".format(vegetation_loss))
plt.imshow(diff_mask, cmap=matplotlib.colors.ListedColormap(["black", "crimson", "silver"], N=None))
plt.axis("off")
patches = [mpatches.Patch(color="crimson", label="vegetation lost")]
plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0)
plt.show()